<a href="https://colab.research.google.com/github/syedasrar88/syedasrar88/blob/main/enterprise_product_last_purchase_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

In [2]:
import datetime
current_date = datetime.datetime.now().date()
month_start_date = datetime.datetime(current_date.year, current_date.month, 1).date()
four_days_ago = current_date - datetime.timedelta(days=4)

In [3]:
yesterday_date_int = int(current_date.strftime("%Y%m%d"))-1

In [4]:
# API URL
api_url = "https://fotproxy.freshontable.com/kpiservices/rest/kpi/getDashboardData"

In [5]:
# request payload
payload = {
    "queryName": "get_entity_achievement_summary",
    "params": {
        "start_period_n": 20240101, #yyyyMMdd
        "end_period_n": yesterday_date_int #yyyyMMdd
    }
}

In [6]:
response = requests.post(api_url, json=payload)
if response.status_code == 200:
    json_data = response.json()
    df = pd.DataFrame(json_data['response'])
else:
    print("Error fetching data from the API:", response.text)

In [7]:
df

,period_id_n,year_n,month_n,day_n,user_id_n,user_name_v,client_id_n,client_name_v,hotel_group_name_v,outlet_name_v,product_id_n,product_name_v,uom_v,image_url_v,incentive_bl,qty_n,amount_n,target_n,incentive_n
0,20240102,2024,1,2,10004,Joyrose Abesamis,54134,MONTAUK BOUTIQUE CAFE AND RESTAURANT LLC,None,Default Outlet,13236,Lettuce Mesclun Mix,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,3.0,135.0,NaN,NaN
1,20240102,2024,1,2,10004,Joyrose Abesamis,54134,MONTAUK BOUTIQUE CAFE AND RESTAURANT LLC,None,Default Outlet,13321,Wild Rocket Lettuce,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,2.0,80.0,NaN,NaN
2,20240102,2024,1,2,10004,Joyrose Abesamis,54134,MONTAUK BOUTIQUE CAFE AND RESTAURANT LLC,None,Default Outlet,13708,Emirati Mix,Box,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,2.0,120.0,NaN,NaN
3,20240102,2024,1,2,10007,Garima Gogia,53986,Madinat Jumeirah LLC-Mas,Jumeirah Hotels,Default Outlet,13059,Fresh Basil leaves,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,0.5,12.5,NaN,NaN
4,20240102,2024,1,2,10007,Garima Gogia,54003,Raw Coffee Company LLC,None,Default Outlet,12105,White Eggs- (Large) 12x30 trays,Box,https://enhancesysdata.s3.amazonaws.com/demo1/...,False,1.0,185.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73900,20240420,2024,4,20,34985,Zeineb Khechimi,54848,MIRAL EXPERIENCES LLC,Miral,Default Outlet,13321,Wild Rocket Lettuce,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,3.0,93.0,50000.0,NaN
73901,20240420,2024,4,20,34985,Zeineb Khechimi,54848,MIRAL EXPERIENCES LLC,Miral,Default Outlet,13364,White Button Mushroom,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,20.0,330.0,50000.0,NaN
73902,20240420,2024,4,20,34985,Zeineb Khechimi,54848,MIRAL EXPERIENCES LLC,Miral,Default Outlet,13560,Zucchini Green,Kg,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,5.0,60.0,50000.0,NaN
73903,20240420,2024,4,20,34985,Zeineb Khechimi,54937,BRIGHT START BEACH RESORT LLC,None,Default Outlet,14296,White Eggs- (Medium) 12x30 trays,Box,https://enhancesysdata.s3.amazonaws.com/fotxua...,False,2.0,350.0,NaN,NaN


In [8]:
df = df[["period_id_n", 'client_name_v', 'product_name_v','uom_v','user_name_v','qty_n']]

In [9]:
df['period_id_n'] = pd.to_datetime(df['period_id_n'], format='%Y%m%d').dt.date

<ipython-input-9-39826672be04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period_id_n'] = pd.to_datetime(df['period_id_n'], format='%Y%m%d').dt.date


In [10]:
df['frequency'] = df.groupby(['client_name_v', 'product_name_v', 'uom_v'])['period_id_n'].transform('nunique')
df = df.sort_values(['period_id_n'])
df

<ipython-input-10-1e0d4ab28540>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frequency'] = df.groupby(['client_name_v', 'product_name_v', 'uom_v'])['period_id_n'].transform('nunique')


,period_id_n,client_name_v,product_name_v,uom_v,user_name_v,qty_n,frequency
0,2024-01-02,MONTAUK BOUTIQUE CAFE AND RESTAURANT LLC,Lettuce Mesclun Mix,Kg,Joyrose Abesamis,3.0,1
454,2024-01-02,HILTON DUBAI PALM JUMEIRAH HOTEL BR OF GOLDEN ...,Capsicum Red,Kg,Divya Gada,20.0,91
455,2024-01-02,HILTON DUBAI PALM JUMEIRAH HOTEL BR OF GOLDEN ...,Capsicum Yellow,Kg,Divya Gada,10.0,88
456,2024-01-02,HILTON DUBAI PALM JUMEIRAH HOTEL BR OF GOLDEN ...,Capsicum Green,Kg,Divya Gada,10.0,90
457,2024-01-02,HILTON DUBAI PALM JUMEIRAH HOTEL BR OF GOLDEN ...,Round Tomato,Kg,Divya Gada,30.0,90
...,...,...,...,...,...,...,...
73366,2024-04-20,WALDORF ASTORIA DUBAI INTERNATIONAL FINANCIAL ...,Shimeji White,Kg,Divya Gada,2.0,41
73367,2024-04-20,WALDORF ASTORIA DUBAI INTERNATIONAL FINANCIAL ...,Mint Pot,Pieces,Divya Gada,2.0,7
73368,2024-04-20,WALDORF ASTORIA DUBAI INTERNATIONAL FINANCIAL ...,Cucumber Long,Kg,Divya Gada,20.0,57
73358,2024-04-20,WALDORF ASTORIA DUBAI INTERNATIONAL FINANCIAL ...,Tendril (Pea Shoots) - Live,Box,Divya Gada,1.0,7


In [11]:
grouped_df = df.groupby(['product_name_v', 'uom_v', 'client_name_v', 'frequency']).agg({
    'period_id_n': 'max',
    'qty_n': 'sum',
    'user_name_v' : 'last'
}).reset_index()

In [12]:
grouped_df

,product_name_v,uom_v,client_name_v,frequency,period_id_n,qty_n,user_name_v
0,71oz Customized Microgreen,Box,SEVENTY ONE STEAK AND GRILL LLC,7,2024-04-04,7.0,Zeineb Khechimi
1,AD White Large Eggs 12x30 - Non Cage Free,Box,Raw Coffee Company LLC,1,2024-01-06,2.0,Garima Gogia
2,AJ White Large Eggs 12x30,Box,PPM CONRAD HOTEL DUBAI,15,2024-04-05,34.0,Divya Gada
3,AJ White Large Eggs 12x30,Box,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,1,2024-01-12,1.0,Divya Gada
4,AJ White Large Eggs 12x30,Box,AL BANDAR INVESTMENT OWNED BY AL AIN AHLIA INS...,3,2024-03-26,3.0,Divya Gada
...,...,...,...,...,...,...,...
5743,camel cheese feta plain W-Oil,Pieces,ARMED FORCES OFFICER CLUB HOTEL,5,2024-04-16,8.0,Garima Gogia
5744,camel cheese feta plain W-Oil,Pieces,Jumeirah Emirates Towers LLC,4,2024-03-11,2.1,Garima Gogia
5745,camel cheese feta plain W-Oil,Pieces,Le Royal Meridien Abu Dhabi Est,1,2024-01-16,1.0,Prachi Kulkarni
5746,camel cheese feta plain W-Oil,Pieces,ROVE HOTELS,1,2024-03-18,2.0,Prachi Kulkarni


In [13]:
sorted_df = grouped_df.sort_values(['period_id_n'], ascending=False)

filtered_df = sorted_df.loc[sorted_df.period_id_n <= four_days_ago].reset_index()
filtered_df['num_of_days_from_last_order'] = current_date - filtered_df.period_id_n
filtered_df

,index,product_name_v,uom_v,client_name_v,frequency,period_id_n,qty_n,user_name_v,num_of_days_from_last_order
0,1676,Fresh Basil leaves,Kg,MIRAL EXPERIENCES LLC,29,2024-04-18,51.00,Zeineb Khechimi,"4 days, 0:00:00"
1,3641,Micro Mix - Sakura,Box,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,45,2024-04-18,61.00,Divya Gada,"4 days, 0:00:00"
2,1508,Fresh Baby Spinach,Kg,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,39,2024-04-18,53.00,Divya Gada,"4 days, 0:00:00"
3,1615,Fresh Basil Pot,Pieces,"THE WB ABU DHABI, CURIO COLLECTION BY HILTON",25,2024-04-18,72.00,Divya Gada,"4 days, 0:00:00"
4,2382,Fresh whole chicken 1100gm,Kg,HONEYCOMB HIFI RESTAURANT LLC,1,2024-04-18,22.00,Prachi Kulkarni,"4 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...
4592,3887,Mozzarella Balls (Fior de Latte 125gm),Kg,Beach Rotana Hotel,1,2024-01-02,0.00,Divya Gada,"111 days, 0:00:00"
4593,4977,Sorrel box (M) 20g,Pieces,Rumba Cuban Bar and Kitchen,1,2024-01-02,2.00,Rian Cruz,"111 days, 0:00:00"
4594,1083,Clam White,Kg,Rumba Cuban Bar and Kitchen,1,2024-01-02,2.00,Rian Cruz,"111 days, 0:00:00"
4595,1556,Fresh Baby Spinach,Kg,LAVO FZ LLC,1,2024-01-02,0.13,Tutku Yagci,"111 days, 0:00:00"


In [14]:
unsold_product_client = filtered_df[['product_name_v','uom_v','client_name_v','user_name_v','frequency','period_id_n','qty_n','num_of_days_from_last_order']]
unsold_product_client.rename(columns = {'product_name_v':'product_name','uom_v':'uom','client_name_v':'client_name','user_name_v':'sales_user','period_id_n':'last_order_date','qty_n':'sum_qty_sold'}, inplace = True)

<ipython-input-14-ba68c528dcaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsold_product_client.rename(columns = {'product_name_v':'product_name','uom_v':'uom','client_name_v':'client_name','user_name_v':'sales_user','period_id_n':'last_order_date','qty_n':'sum_qty_sold'}, inplace = True)


In [15]:
unsold_product_client

,product_name,uom,client_name,sales_user,frequency,last_order_date,sum_qty_sold,num_of_days_from_last_order
0,Fresh Basil leaves,Kg,MIRAL EXPERIENCES LLC,Zeineb Khechimi,29,2024-04-18,51.00,"4 days, 0:00:00"
1,Micro Mix - Sakura,Box,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,Divya Gada,45,2024-04-18,61.00,"4 days, 0:00:00"
2,Fresh Baby Spinach,Kg,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,Divya Gada,39,2024-04-18,53.00,"4 days, 0:00:00"
3,Fresh Basil Pot,Pieces,"THE WB ABU DHABI, CURIO COLLECTION BY HILTON",Divya Gada,25,2024-04-18,72.00,"4 days, 0:00:00"
4,Fresh whole chicken 1100gm,Kg,HONEYCOMB HIFI RESTAURANT LLC,Prachi Kulkarni,1,2024-04-18,22.00,"4 days, 0:00:00"
...,...,...,...,...,...,...,...,...
4592,Mozzarella Balls (Fior de Latte 125gm),Kg,Beach Rotana Hotel,Divya Gada,1,2024-01-02,0.00,"111 days, 0:00:00"
4593,Sorrel box (M) 20g,Pieces,Rumba Cuban Bar and Kitchen,Rian Cruz,1,2024-01-02,2.00,"111 days, 0:00:00"
4594,Clam White,Kg,Rumba Cuban Bar and Kitchen,Rian Cruz,1,2024-01-02,2.00,"111 days, 0:00:00"
4595,Fresh Baby Spinach,Kg,LAVO FZ LLC,Tutku Yagci,1,2024-01-02,0.13,"111 days, 0:00:00"


In [16]:
unsold_product_client.to_csv('data.csv') # to download the above data in a csv execute this cell

In [17]:
last_sold_four_days_ago = unsold_product_client.loc[unsold_product_client.last_order_date == four_days_ago]
last_sold_four_days_ago.reset_index()

,index,product_name,uom,client_name,sales_user,frequency,last_order_date,sum_qty_sold,num_of_days_from_last_order
0,0,Fresh Basil leaves,Kg,MIRAL EXPERIENCES LLC,Zeineb Khechimi,29,2024-04-18,51.0,"4 days, 0:00:00"
1,1,Micro Mix - Sakura,Box,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,Divya Gada,45,2024-04-18,61.0,"4 days, 0:00:00"
2,2,Fresh Baby Spinach,Kg,YAS BAY HOTEL LLC BY HILTON MANAGEMENT - SOLE...,Divya Gada,39,2024-04-18,53.0,"4 days, 0:00:00"
3,3,Fresh Basil Pot,Pieces,"THE WB ABU DHABI, CURIO COLLECTION BY HILTON",Divya Gada,25,2024-04-18,72.0,"4 days, 0:00:00"
4,4,Fresh whole chicken 1100gm,Kg,HONEYCOMB HIFI RESTAURANT LLC,Prachi Kulkarni,1,2024-04-18,22.0,"4 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...
210,210,Beef Tomatoes,Kg,HILTON DUBAI PALM JUMEIRAH HOTEL BR OF GOLDEN ...,Divya Gada,39,2024-04-18,303.0,"4 days, 0:00:00"
211,211,Baby Kale,Kg,Jumeirah Restaurants LLC (Noodle House CityWalk),Garima Gogia,21,2024-04-18,26.0,"4 days, 0:00:00"
212,212,Portobello Mushroom,Kg,Celebrating Moments,Garima Gogia,13,2024-04-18,35.5,"4 days, 0:00:00"
213,213,Red Amaranth Cress,Box,HILTON DUBAI JUMREIRAH,Divya Gada,3,2024-04-18,2.0,"4 days, 0:00:00"
